## Veri Ön İşleme (Sklearn Pandas)

Sklearn-Pandas paketini kullanarak verilerinizi çok daha pratik bir şekilde işlemenize yardımcı olacaktır.Elinize gelen veri bir çok açıdan kusurlu olabilmektedir.Bu kusurlar eksik veriler, aykırı değerler yada kendini tekrar eden veriler gibi bir çok şey olabilir.Aynı zaman bu paket ile birlikte ölçeklendirme işlemleri gibi bir çok şeyi aynı anda yapabiliriz.

Sklearn-pandas paketinin ne olduğunu nasıl ön işleme adımlarını gerçekleştirebildiğimize bakalım ardından gerçek bir veri seti üzerinde bu adımları gerçekleştiriyor olacağız.

In [ ]:
# Kurulum
! pip install sklearn-pandas

Sklearn-pandas uygulamalarını gerçekleştirebileceğimiz basit bir veri seti oluşturalım ve bunun üzerinde bazı adımları gerçekleştirmeyi deneyelim.

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

data = pd.DataFrame({'pet':      ['cat', 'dog', 'dog', 'fish', 'cat', 'dog', 'cat', 'fish'],
                     'children': [4., 6, 3, 3, 2, 3, 5, 4],
                     'salary':   [90., 24, 44, 27, 32, 59, 36, 27]})


``sklearn-pandas`` içerisinde bulunan **DataFrameMapper** ile birlikte bir pipeline yapısı kurarak verilerimiz üzerinde ön işleme adımları gerçekleştirebiliriz.Ayrı ayrı değişkenler üzerinde veya değişkenleri bir liste halinde vererek üzerinde ön işleme adımı gerçekleştirebiliriz.Peki bu nasıl olacak görelim!

In [3]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelBinarizer , StandardScaler

mapper = DataFrameMapper([
     ('pet', LabelBinarizer()),
     (['children'], StandardScaler())])

In [4]:
mapper.fit_transform(data)

array([[ 1.        ,  0.        ,  0.        ,  0.20851441],
       [ 0.        ,  1.        ,  0.        ,  1.87662973],
       [ 0.        ,  1.        ,  0.        , -0.62554324],
       [ 0.        ,  0.        ,  1.        , -0.62554324],
       [ 1.        ,  0.        ,  0.        , -1.4596009 ],
       [ 0.        ,  1.        ,  0.        , -0.62554324],
       [ 1.        ,  0.        ,  0.        ,  1.04257207],
       [ 0.        ,  0.        ,  1.        ,  0.20851441]])

 Veri setlerimiz içerisinde tarih belirten değişkenler oldukça sık karşımıza çıkabiliyor.Bunun gibi durumlarda sklearn-pandas paketinden yararlanabiliriz.Sadece bulunduğumuz veri seti için değil aynı zamanda ilerde karşımıza çıkacak veri setlerindeki tarih formatındaki tüm değişkenlerde uygulabiliriz.

In [5]:
from sklearn.base import TransformerMixin
class DateEncoder(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dt = X.dt
        return pd.concat([dt.year, dt.month, dt.day], axis=1)
    
   
dates_df = pd.DataFrame(
    {'dates': pd.date_range('2015-10-30', '2015-11-02')})    

 > DataFrameMapper parametresi olan **input_df** True olarak kabul etmemiz gerekmektedir.Bunun sebebi sklearn transformatörlerin tarihsel değişkenler üzerinde pandas DataFrame ile değil numpy dizileri ile çalışıyor olmasındandır.

In [6]:
mapper_dates = DataFrameMapper([
     ('dates', DateEncoder())],  input_df=True)

mapper_dates.fit_transform(dates_df)

array([[2015,   10,   30],
       [2015,   10,   31],
       [2015,   11,    1],
       [2015,   11,    2]], dtype=int64)

Bu zamana kadar yaptığımız ön işlemlerde her zaman numpy dizisi olarak çıktı alımı gerçekleştirdik fakat pandas DataFrame olarak çıktı da elde edebiliriz.Bunun için DataFrameMapper parametresi olan **df_out = True** olarak girmemiz gerekmektedir.

In [7]:
mapper = DataFrameMapper([
     ('pet', LabelBinarizer()),
     (['children'], StandardScaler())] , df_out=True)
mapper.fit_transform(data)

,pet_cat,pet_dog,pet_fish,children
0,1,0,0,0.208514
1,0,1,0,1.876630
2,0,1,0,-0.625543
3,0,0,1,-0.625543
4,1,0,0,-1.459601
5,0,1,0,-0.625543
6,1,0,0,1.042572
7,0,0,1,0.208514


Veriler üzerinde ön işleme adımı gerçekleştirirken DataFrameMapper hattı içerisinde veri setinden kaldırılmasını istediğimiz değişkeni belirtebiliriz.Bunun için ``drop_cols = ['col',...]`` argümanını kullanmamız gerekmektedir.

In [8]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
data= pd.DataFrame({'gender': ['male', 'male', 'female', 'male', 'female', 'female', 'male','female'],
                     'age': [15, 20, 40, 30, 55, 78, 12, 90],
                     'salary':   [1200, 3000, 4400, 2700, 3200, 5900, 1000,4000 ]})

mapper = DataFrameMapper([(['gender'], OneHotEncoder()),
                          (['age'] , MinMaxScaler()),
                          ], drop_cols=['salary'] , df_out=True)
mapper.fit_transform(data)

,gender_x0_female,gender_x0_male,age
0,0.0,1.0,0.038462
1,0.0,1.0,0.102564
2,1.0,0.0,0.358974
3,0.0,1.0,0.230769
4,1.0,0.0,0.551282
5,1.0,0.0,0.846154
6,0.0,1.0,0.000000
7,1.0,0.0,1.000000


> Bazı ön işleme adımlarında girdi olarka 2 boyutlu almaktadır.Ölçeklendirme işlemleri buna bir örnektir.Bu durumda DateFrameMapper içerisinde değişkenimizi belirtirken liste içerisinde belirtmemiz gerekmektedir.

Aynı anda birden fazla değişken üzerinde işlemlerimizi uygulayabiliriz.Değişkenlerimizi önceki belirttiğimiz şeklinde tek tek değilde bir liste içerisinde belirterek üzerinde ortak bir işlem gerçekleştirebiliriz.

In [9]:
from sklearn.decomposition import PCA
mapper2 = DataFrameMapper([
     (['age', 'salary'], PCA(n_components=1))
 ], df_out=True)

mapper2.fit_transform(data)

,age_salary
0,-1975.191335
1,-175.303020
2,1224.839829
3,-475.129942
4,25.175686
5,2725.229161
6,-2175.213779
7,825.593401


Bir liste içerisinde uygulayacağımız transformatörleri belirterek sırası ile sütuna aynı anda işlemler uygulatabiliriz.

In [25]:
data2 = pd.DataFrame({'age': [15, 20, np.nan, 30, np.nan, 78, 12, np.nan]})
data2

,age
0,15.0
1,20.0
2,NaN
3,30.0
4,NaN
5,78.0
6,12.0
7,NaN


Birden fazla değişken üzerinden aynı işlemleri yapabildiğimiz gibi birden fazla işlemde gerçekleştirebiliriz.Yapacağımız işlemleri bir liste içerisinde tanımlamamız yeterli olacaktır.Unutmayalım ki liste içerisinde işlemleri nasıl tanımlarsanız gerçekleşme sırasıda öyle gerçekleşecektir.

In [11]:
from sklearn.impute import SimpleImputer
transformers = [SimpleImputer() , StandardScaler()]

mapper2 = DataFrameMapper([(['age'] , transformers)] ,df_out=True).fit_transform(data2)

> *age* sütununa sırası ile önce eksik değerleri doldurma işlemi gerçekleştirip ardından standardizasyon işlemi gerçekleştirmiştir.

Veri setimiz üzerindeki bazı değişkenler üzerinde herhangi bir işlem uygulanmamasını isteyebiliriz.Bu gibi durumlarda istemediğimiz değişkeni ('col' , **None**) olarak belirtirsek herhangi bir işlem gerçekleşmeyecektir.

In [12]:
data3 = pd.DataFrame({'gender': ['male', 'male', 'female', 'male', 'female', 'female', 'male','female'],
                     'age': [15, 20, 40, 30, 55, 78, 12, 90],
                     'salary':   [1200, 3000, 4400, 2700, 3200, 5900, 1000,4000 ]})

mapper3 = DataFrameMapper([(['age' , 'salary'] ,StandardScaler()) , ('gender' , None)], df_out=True)
mapper3.fit_transform(data3)

,age_salary_0,age_salary_1,gender
0,-1.001823,-1.298928,male
1,-0.819673,-0.115095,male
2,-0.091075,0.805664,female
3,-0.455374,-0.312400,male
4,0.455374,0.016442,female
5,1.293263,1.792191,female
6,-1.111113,-1.430465,male
7,1.730422,0.542590,female


DataFrameMapper argüman olarak ``default`` almaktadır.Bu argümana belirtmediğimiz değişkenler üzerinde uygulanmasını istediğimiz işlemleri iletebiliriz.

In [13]:
data = pd.DataFrame({'pet':      ['cat', 'dog', 'dog', 'fish', 'cat', 'dog', 'cat', 'fish'],
                    'children': [4., 6, 3, 3, 2, 3, 5, 4],
                    'salary':   [90., 24, 44, 27, 32, 59, 36, 27]})

mapper4 = DataFrameMapper([
     (['pet'], LabelBinarizer()),
     ('children', None) ],  default=StandardScaler())
mapper4.fit_transform(data)

array([[ 1.        ,  0.        ,  0.        ,  4.        ,  2.27500192],
       [ 0.        ,  1.        ,  0.        ,  6.        , -0.87775665],
       [ 0.        ,  1.        ,  0.        ,  3.        ,  0.07762474],
       [ 0.        ,  0.        ,  1.        ,  3.        , -0.73444944],
       [ 1.        ,  0.        ,  0.        ,  2.        , -0.49560409],
       [ 0.        ,  1.        ,  0.        ,  3.        ,  0.79416078],
       [ 1.        ,  0.        ,  0.        ,  5.        , -0.30452782],
       [ 0.        ,  0.        ,  1.        ,  4.        , -0.73444944]])

Bazen aynı dönüşümü birkaç DataFrame sütununa uygulamak gerekir. Bu işlemi basitleştirmek için paket, bir sütun listesi ve özellik dönüştürücü sınıfı (veya sınıf listesi) kabul eden ve DataFrameMapper tarafından kabul edilebilir bir özellik tanımı üreten **gen_features** işlevi sağlar.

İki argüman almaktadır;
* columns, değişkenlerimiz liste içerisinde belirtilir.
* classes, uygulanacak transformatör liste içerisinde belirtilir.

In [26]:
from sklearn_pandas import gen_features
from sklearn.preprocessing import LabelEncoder

feature_def = gen_features(
     columns=['col1', 'col2', 'col3'],
     classes=[LabelEncoder]
 )

mapper5 = DataFrameMapper(feature_def , df_out=True)

data5 = pd.DataFrame({
     'col1': ['yes', 'no', 'yes'],
     'col2': [True, False, False],
     'col3': ['one', 'two', 'three']
})
data5

,col1,col2,col3
0,yes,True,one
1,no,False,two
2,yes,False,three


In [15]:
mapper5.fit_transform(data5)

,col1,col2,col3
0,1,1,0
1,0,0,2
2,1,0,1


Buraya kadarki süreçte sklearn_pandas paketinin içerisinde bulunan ``DataFrameMapper`` işlevi üzerinde durmuş olduk fakat sadece bunun ile kalmamaktadır.Sklearn kütüphanesi içerisinde bulunan SimpleImputer gibi sklearn_pandas içerisinde de eksik değerleri doldurabileceğimiz veya dönüştürme işlemlerini gerçekleştirebileceğimiz işlevler bulunmaktadır.

Şimdi bu zamana kadar gerçekleştirdiğimiz işlemleri gerçek bir veri seti üzerinde deneyelim.270K kullanıcı tarafından derecelendirilen 45K film içeren Kaggle'ın “Film Veri Kümesini” kullanacağım. Veri setini [buradan](https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings.csv) indirebilirsiniz.




In [16]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn_pandas import gen_features , DataFrameMapper

def organise_dataset(path):
    dataset = pd.read_csv(path)

    dataset.rename(columns={'id': 'movieId'}, inplace=True)

    dataset['movieId'] = dataset['movieId'].apply(lambda x: x if x.isdigit() else 999999999)
    dataset['budget'] = dataset['budget'].apply(lambda x: x if x.isdigit() else 0)
    dataset['movieId'] = dataset['movieId'].astype('int64')

    dataset = dataset.drop(['overview','homepage','original_title','imdb_id', 'belongs_to_collection', 'genres','poster_path', 'production_companies','production_countries','spoken_languages', 'tagline'], axis=1)
    col_cat_list = list( dataset.select_dtypes(exclude=np.number) )
    col_num_list = list( dataset.select_dtypes(include=np.number) )
    col_none = ['movieId']

    num_cols = ['budget', 'popularity']
    [ col_cat_list.remove(x) for x in num_cols ]
    [ col_num_list.append(x) for x in num_cols ]
    col_cat_list.remove('release_date')
    col_num_list.remove('movieId')

    dataset[col_cat_list] = dataset[col_cat_list].astype('category')
    # dataset[col_num_list] = dataset[col_num_list].astype('float64', errors='coerce')
    dataset['budget'] = pd.to_numeric(dataset['budget'],errors="coerce")
    dataset['popularity'] = pd.to_numeric(dataset['popularity'],errors="coerce")

    # Convert to list of lists
    col_categorical = [ [x] for x in col_cat_list ]
    col_numerical   = [ [x] for x in col_num_list ]
    col_date = ['release_date']

    dataset['release_date'] = pd.to_datetime(dataset['release_date'], errors="coerce")

    return dataset , col_categorical , col_numerical , col_date, col_none

movies , col_categorical , col_numerical , col_date , col_none = organise_dataset("movies_metadata.csv")

**Veri setimiz üzerinde ön işleme adımı gerçekleştirmeden önce düzenlemeler gerçekleştirilmiştir.**
* Veri tipi uyumsuzluğunun giderilmesi.Bazı değişkenler sayısal değerler olmasına rağmen kategorik tipinde gözükmekteydiler.
* Ön işleme adımında kullanmayacağımız değişkenlerin kaldırılması
* Değişkenlerin liste halinde çekilmesidir.


In [17]:
pd.DataFrame(movies.isnull().sum().sort_values() , columns=['missing_values'])

,missing_values
adult,0
budget,0
movieId,0
popularity,6
revenue,6
title,6
video,6
vote_average,6
vote_count,6
original_language,11


In [18]:
from sklearn.base import TransformerMixin

class DateEncoder(TransformerMixin):
      def fit(self, X, y=None):
        return self

      def transform(self, X):
        X = pd.to_datetime(movies['release_date'])
        dt = X.dt
        return pd.concat([dt.year, dt.month, dt.day],  axis=1)

Tarih değerlerini tutan ``release_date`` değişkenimizi yıl , ay ve gün formatına dönüştürecek olan sınıf yapısı tanımlanmıştır.

**Veri setimiz için ön işleme adımlarına geçebiliriz;**

* İlk olarak yapacağımız ``release_date`` değişkenini Datetime formatına getirmelidir.Yukarıda bunun için bir transformatör oluşturup ardından DateTimeMapper üzerinden işlemi gerçekleştirebiliriz.

* Eksik değerlerimizi doldurduktan sonra veri setimizi ölçeklendirme işlemi gerçekleştirebiliriz.Kullanacağımız algoritmalar KNN gibi mesafe dayalı algoritmalar ise veri setimizi ölçeklendirmek çok işimize yarayacaktır.
* Kategorik değişkenler üzerinde eksik değer doldurma ve encoding işlemi gerçekleştireceğiz.

In [19]:
classes_categorical_preprocessing = [ {'class':SimpleImputer, 'strategy' : 'most_frequent'}, LabelEncoder]
classes_numerical_preprocessing = [ {'class':SimpleImputer, 'strategy' : 'median'}, StandardScaler]
classes_date_preprocessing = [DateEncoder]
classes_none_preprocessing = [None]

Uygulacağımız işlemleri liste içerisinde belirttik.``gen_features`` birden fazla değişkene birden fazla işlem uygulama imkanı vermektedir.

- **movieId** değişkenimiz üzerinde herhangi bir ön işleme gerçekleştirmek istemediğimizden **None** değerine sahip bir değişken tanımlanmıştır.


In [20]:
# Main Gen-Features
feature_gen = gen_features(columns = [] , classes = [])

# Categorical Variables
feature_def_categorical = gen_features(
    columns = col_categorical
    , classes = classes_categorical_preprocessing
)

# Numerical Variables
feature_def_numerical = gen_features(
    columns = col_numerical
    , classes = classes_numerical_preprocessing
)

# Date Variables
feature_def_date = gen_features(
    columns = col_date
    , classes = classes_date_preprocessing
)

# Not Apply Variables
feature_def_none = gen_features(
    columns = col_none
    , classes = classes_none_preprocessing
)

feature_gen.extend(feature_def_categorical)
feature_gen.extend(feature_def_date)
feature_gen.extend(feature_def_numerical)
feature_gen.extend(feature_def_none)

Veri setimize uygulacağımız gen_features işlevlerini tanımladık.Artık son adımlara yaklaşıyoruz.DateTimeMapper kullanarak bunları veri setinde uygulamak kaldı.
> Boş bir gen_features nesnesi oluşturarak üzerinden extend işlemi gerçekleştirdik.Bunun anlamı bütün gen_features birleştir demektir.**extend** yaparken işlemlerin sırasına göre yapılması gerekmektedir.

In [23]:
feature_gen

[(['adult'], [SimpleImputer(strategy='most_frequent'), LabelEncoder()], {}),
 (['original_language'],
  [SimpleImputer(strategy='most_frequent'), LabelEncoder()],
  {}),
 (['status'], [SimpleImputer(strategy='most_frequent'), LabelEncoder()], {}),
 (['title'], [SimpleImputer(strategy='most_frequent'), LabelEncoder()], {}),
 (['video'], [SimpleImputer(strategy='most_frequent'), LabelEncoder()], {}),
 ('release_date', [<__main__.DateEncoder at 0x12607b987c0>], {}),
 (['revenue'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 (['runtime'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 (['vote_average'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 (['vote_count'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 (['budget'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 (['popularity'], [SimpleImputer(strategy='median'), StandardScaler()], {}),
 ('movieId', None, {})]

In [22]:
final_movies= DataFrameMapper(feature_gen , df_out = True).fit_transform(movies)
final_movies.rename(columns = {'release_date_0':'year', 'release_date_1':'month' , 'release_date_2' : 'day'}, inplace = True)

In [24]:
final_movies.head(10)

,adult,original_language,status,title,video,year,month,day,revenue,runtime,vote_average,vote_count,budget,popularity,movieId
0,3,20,4,39018,0,1995.0,10.0,30.0,5.632841,-0.342939,1.081946,10.798693,1.479373,3.168317,862
1,3,20,4,16806,0,1995.0,12.0,15.0,3.911075,0.257643,0.666161,4.688049,3.488165,2.347099,8844
2,3,20,4,13371,0,1995.0,12.0,22.0,-0.174232,0.179307,0.458269,-0.036404,-0.242450,1.464059,15602
3,3,20,4,40428,0,1995.0,12.0,22.0,1.091980,0.858225,0.250376,-0.154464,0.675855,0.156246,31357
4,3,20,4,11199,0,1995.0,2.0,10.0,1.016224,0.309868,0.042484,0.128474,-0.242450,0.910290,11862
5,3,20,4,13962,0,1995.0,12.0,15.0,2.739562,1.981052,1.081946,3.615327,3.201195,2.498537,949
6,3,20,4,26420,0,1995.0,12.0,15.0,-0.174232,0.858225,0.302349,0.063337,3.086407,0.625486,11860
7,3,20,4,38807,0,1995.0,12.0,22.0,-0.174232,0.074858,-0.113435,-0.132073,-0.242450,-0.059964,45325
8,3,20,4,29316,0,1995.0,12.0,22.0,0.826122,0.309868,-0.061462,0.130509,1.766343,0.384737,9091
9,3,20,4,13044,0,1995.0,11.0,16.0,5.300790,0.936562,0.510242,2.206744,3.086407,1.959171,710


Sklearn-pandas paketini kullanarak çok daha pratik bir şekilde veri ön işleme adımlarını gerçekleştirmiş olduk.